In [ ]:
#Import libraries need for data manipulation and visualization

import pandas as pd
import numpy as np
import re
import os
import plotly.express as px

# 1. Phenophase Data

## 1.1 Import Phenophase Data

This section uses the same data from Milestone 1 - Section 2: Download Phenophase Data and follows similar cleaning steps.

Data is from https://data.usanpn.org/observations/ and the csv file was added to this project.

Phenophases we are interested in:

(descriptions are from https://www.usanpn.org/files/shared/files/Plant%20and%20Animal%20Phenophase%20Definition%20Supplement.pdf)

First Leaf - (aka breaking leaf buds) "One or more breaking leaf buds are visible on the plant." For lilacs in particular, "In at least 3 locations on the plant, a breaking leaf bud is visible."

First bloom - (aka open flowers) "One or more open, fresh flowers are visible on the plant." For lilacs in particular, "at least half (50%) of the flower clusters have at least one open fresh flower. The lilac flower cluster is a grouping of many, small individual flowers."

In [ ]:
#read in phenometrics data
phen = pd.read_csv('individual_phenometrics_data.csv')

#column definitions can be found here: 
#"Individual Phenometrics Datafield Descriptions" https://data.usanpn.org/observations and then click on Metadata
columns_to_keep = ['Site_ID', 'Individual_ID', 'Latitude', 'Longitude','Phenophase_Description', 'First_Yes_Year',
                   'First_Yes_Month', 'First_Yes_Day', 'First_Yes_DOY', 'Genus', 'State']

phen = phen[phen.columns[phen.columns.isin(columns_to_keep)]]

#Narrowing down phenophases of interest and
#Update the phenophase names to help merge the historical with the more recent observations

#remove portion of name in parentheses
phen['Phenophase_Description'] = phen['Phenophase_Description'].apply(lambda x: x.split(' (')[0])

#replace more recent phenophase terms to be consistent with historical terms
phen['Phenophase_Description'] = phen['Phenophase_Description'].replace(
            ['End of flowering','Full flowering','Open flowers', 'Breaking leaf buds'],
            ['End bloom','Full bloom', 'First bloom', 'First leaf'])

#filter to just get the phenophases we are interested in 
temp = ['First leaf', 'First bloom']
phen = phen[phen['Phenophase_Description'].isin(temp)]

#Defining Regions
northeast = ['ME', 'VT', 'NH', 'MA', 'RI', 'CT', 'NJ', 'DE', 'MD', 'PA', 'NY']
southeast = ['VA, NC, SC, GA, FL, AL']

phen['Region'] = np.where(phen['State'].isin(northeast), 'Northeast','Southeast')

#need to remove Site_ID 26116. We discovered in Milestone 1, it is geolocated to Kansas but is actually in Maine. 
phen.drop(phen[phen['Site_ID'] ==26116].index, inplace=True)

##SLICING
#subset to just lilacs
phen = phen[phen['Genus']=='Syringa']

#only using Northeast for this project
phen = phen[phen['Region']=='Northeast']

#rename columns for easier use
phen = phen.rename(columns = {'First_Yes_Year':'year', 'First_Yes_DOY':'DOY'})

#only include years 1980-2020 to match with weather data
phen = phen[phen['year'].between(1980,2020)]

In [ ]:
phen

,Site_ID,Latitude,Longitude,State,Genus,Individual_ID,Phenophase_Description,year,First_Yes_Month,First_Yes_Day,DOY,Region
5913,209,43.799999,-72.269997,VT,Syringa,3475,First leaf,1980,5,2,123,Northeast
5914,209,43.799999,-72.269997,VT,Syringa,3475,First bloom,1980,5,21,142,Northeast
5918,177,43.970085,-74.217880,NY,Syringa,3521,First leaf,1980,4,30,121,Northeast
5920,65,41.972836,-73.220451,CT,Syringa,3906,First leaf,1980,4,22,113,Northeast
5921,65,41.972836,-73.220451,CT,Syringa,3906,First bloom,1980,5,21,142,Northeast
...,...,...,...,...,...,...,...,...,...,...,...,...
276853,38398,43.027870,-75.979095,NY,Syringa,240184,First leaf,2020,5,18,139,Northeast
277131,38956,41.802170,-70.556267,MA,Syringa,242065,First bloom,2020,5,18,139,Northeast
277210,39049,41.800629,-69.969147,MA,Syringa,242883,First bloom,2020,5,23,144,Northeast
279019,40959,42.921581,-78.885521,NY,Syringa,249679,First bloom,2020,9,25,269,Northeast


Important note: Site_ID refers to the site where data is recorded.  Individual_ID refers to the specific plant.  There can be multiple individuals at one site.  

In [ ]:
#How many unique site locations are there? How many unique plant ids are there? 

#number of locations 
print("number of unique locations:", len(phen.groupby(['Latitude', 'Longitude']).count()))
#number of plants
print("number of unique plants:", len(phen.groupby(['Individual_ID']).count())) #636
#number of sites
print("number of unique sites:", len(pd.unique(phen['Site_ID']))) 

number of unique locations: 434
number of unique plants: 540
number of unique sites: 437


In [ ]:
#find unique latitude and longitude (latlon) locations
latlon_df = phen.drop_duplicates(subset=['Latitude', 'Longitude', 'Site_ID'])[['Latitude', 'Longitude', 'Site_ID']]

#check which latlon have multiple sites
latlon_df.groupby(['Latitude', 'Longitude']).count().sort_values('Site_ID', ascending=False)

,,Site_ID
Latitude,Longitude,
40.635593,-75.352516,2
43.380001,-72.599998,2
42.466228,-76.485329,2
38.630001,-75.470001,1
43.337627,-70.548935,1
...,...,...
41.830002,-75.870003,1
41.817360,-71.464653,1
41.810211,-71.449440,1


Looks like there are 3 latlons that have 2 sites at each.

## 1.2 Phenophase Data Checks

Additional data checks for this Milestone Project.  Checking to see if there is only one phenophase description for each plant per year. For example, there should not be multiple dates for "first leaf" on the same plant in the same year.

In [ ]:
#function to check for mulitple dates for a phenophase type 
#by grouping each plant for each year and counting number of entries

def mult_entry_phen(df, phen):
    #subset by phenophase
    phendf = df[df['Phenophase_Description']==phen]
    #count the number of entries there are for DOY for each year and indivdual plant
    mult = phendf.groupby(['year', 'Individual_ID'])['DOY'].count()
    #return series where indivdual plants have counts greater than 1
    #using the 'greater than' gt() method
    return(mult[mult.gt(1)].reset_index())


In [ ]:
mult_firstleaf = mult_entry_phen(phen,'First leaf')
mult_firstleaf

,year,Individual_ID,DOY
0,2009,1490,2
1,2010,414,2
2,2010,2098,2
3,2010,3592,3
4,2010,4118,2
5,2011,947,2
6,2012,3115,2
7,2012,4118,4
8,2012,5073,2
9,2012,17622,2


In [ ]:
mult_firstbloom = mult_entry_phen(phen,'First bloom')
mult_firstbloom

,year,Individual_ID,DOY
0,2010,3115,2
1,2010,4118,3
2,2011,946,2
3,2011,4118,2
4,2011,5072,2
5,2011,11263,2
6,2011,12483,2
7,2012,4118,3
8,2012,16062,2
9,2012,16063,2


The above 2 cells show that there are several individual plants that have multiple days listed as first leaf date and first bloom date within the same year.  Need to remove those individuals for those particular years.

In [ ]:
#create function
def drop_id_yr(df_wrongval, maindf):
    #create list of lists to find individual id and year
    bad_lst = df_wrongval[['Individual_ID', 'year']].values.tolist()
    to_drop_idx = []
    for i in range(len(bad_lst)):
        temp = maindf.query('Individual_ID=='+str(bad_lst[i][0])+'and year=='+str(bad_lst[i][1]))
        temp_idx = temp.index.tolist()
        for i in temp_idx:
            to_drop_idx.append(i)
    #remove from main dataframe based on index saved in to_drop_idx
    maindf = maindf[~maindf.index.isin(to_drop_idx)]
    return maindf

In [ ]:
phen = drop_id_yr(mult_firstleaf, phen)

In [ ]:
phen = drop_id_yr(mult_firstbloom, phen)

In [ ]:
phen

,Site_ID,Latitude,Longitude,State,Genus,Individual_ID,Phenophase_Description,year,First_Yes_Month,First_Yes_Day,DOY,Region
5913,209,43.799999,-72.269997,VT,Syringa,3475,First leaf,1980,5,2,123,Northeast
5914,209,43.799999,-72.269997,VT,Syringa,3475,First bloom,1980,5,21,142,Northeast
5918,177,43.970085,-74.217880,NY,Syringa,3521,First leaf,1980,4,30,121,Northeast
5920,65,41.972836,-73.220451,CT,Syringa,3906,First leaf,1980,4,22,113,Northeast
5921,65,41.972836,-73.220451,CT,Syringa,3906,First bloom,1980,5,21,142,Northeast
...,...,...,...,...,...,...,...,...,...,...,...,...
276772,38658,44.145035,-68.851357,ME,Syringa,239777,First leaf,2020,4,29,120,Northeast
276851,38398,43.027870,-75.979095,NY,Syringa,240184,First bloom,2020,5,20,141,Northeast
276853,38398,43.027870,-75.979095,NY,Syringa,240184,First leaf,2020,5,18,139,Northeast
277131,38956,41.802170,-70.556267,MA,Syringa,242065,First bloom,2020,5,18,139,Northeast


Would like to see the distribution of the DOY before we continue.

In [ ]:
px.scatter(phen, x='year', y='DOY', color='Phenophase_Description')


In [ ]:
#remove those where DOY greater than 200 
#August for first leaf doesn't make sense and not currently investigating fall blooms
phen_base = phen[phen['DOY']<200]

## 1.3 Phenophase Interval

We would like to investigate the interval between first leaf and first bloom. The below code creates a data frame for just looking at the interval.

In [ ]:
#first of all, do all plants have first leaf and first bloom?
#for each year, does each plant id have 2 values (DOY)?
df = phen_base.groupby(['year', 'Individual_ID'])['DOY'].count().reset_index()
df.groupby('DOY').count()

,year,Individual_ID
DOY,,
1,491,491
2,1648,1648


1674 plants have both bloom and leaf values, 491 have one or the other.  When we create the data frame below, we will drop those observations that do not create an interval for the year.

In [ ]:
#we only need certain columns for the interval, so creating a subset
phen_int = phen_base[['Individual_ID', 'Phenophase_Description', 'year', 'DOY']]

#pivoting into wide format so each row is a single plant's info for the year
phen_int = pd.pivot_table(phen_int, values='DOY', index=['Individual_ID', 'year'], columns='Phenophase_Description').reset_index()

#calculating interval
phen_int['interval'] = phen_int['First bloom'] - phen_int['First leaf']

#only keeping the columns we need
phen_int = phen_int[['Individual_ID', 'year', 'interval']]


In [ ]:
#let's view the intervals
phen_int.sort_values('interval')

Phenophase_Description,Individual_ID,year,interval
85,1669,2010,1.0
2136,240184,2020,2.0
45,1423,2009,3.0
1204,56187,2001,4.0
820,56082,1982,4.0
...,...,...,...
2133,237790,2020,NaN
2134,238151,2020,NaN
2135,239777,2020,NaN
2137,242065,2020,NaN


We have several negative values, several zero values, and then many NaN values.  But also many positive values for the interval, which are the values we are interested in exploring.

In [ ]:
#concern about negative value, looks like they switched first leaf and first bloom on those individuals for those years

#need to remove from main phen_base dataframe
phen_base = drop_id_yr(phen_int.query('interval<0'), phen_base)

#and remove from phen_int dataframe
phen_int = drop_id_yr(phen_int.query('interval<0'), phen_int)

In [ ]:
#concern about those with value of 0, does that mean bloom and leaf happened same day? 
#YES, remove those as that must be a recording error, so not sure what date is for what phenophase
#separate out the ones with an interval of zero
# zero_int = phen_int[phen_int['interval']==0]

#need to remove from main phen_base dataframe
# phen_base = drop_id_yr(zero_int, phen_base)

#and remove from phen_int dataframe as well as remove NaN
phen_int = phen_int[phen_int['interval']>0]


In [ ]:
phen_int

Phenophase_Description,Individual_ID,year,interval
0,414,2009,21.0
1,414,2011,20.0
2,414,2012,28.0
3,521,2009,34.0
4,521,2010,29.0
...,...,...,...
2109,196967,2020,58.0
2112,202475,2020,43.0
2125,235117,2020,11.0
2130,236586,2020,36.0


In [ ]:
#need site id to be able to join to weather data
#create dictionary of which individual ids are in which sites
#use the main phen_base df
#we need DOY too for the weather
sites_indiv = phen_base.drop_duplicates(subset=['Site_ID', 'Individual_ID']).reset_index()
#need to keep where pheno is FIRST BLOOM so we join weather data on the first bloom date
sites_indiv = sites_indiv[sites_indiv['Phenophase_Description']=='First bloom']
sites_indiv = sites_indiv[['Site_ID', 'Individual_ID', 'DOY', 'year']]

# #need to join on indiv id and year
phen_int = phen_int.merge(sites_indiv, on=['Individual_ID', 'year'])
phen_int

,Individual_ID,year,interval,Site_ID,DOY
0,414,2009,21.0,44,121
1,521,2009,34.0,2,140
2,943,2009,24.0,952,126
3,975,2009,19.0,974,121
4,1197,2009,17.0,507,124
...,...,...,...,...,...
117,196967,2019,17.0,34052,145
118,202475,2020,43.0,34675,150
119,235117,2020,11.0,38132,125
120,236586,2020,36.0,38404,145


In [ ]:
#save csv
phen_base.to_csv('phen_base.csv', index=False)

# 2. Weather Data

## 2.1 Download Weather Data

We've chosen to use a gridded weather dataset rather than weather stations with surface observed variables.  The Daymet dataset is computed from ground observations but the values are interpolated between the stations to create a full coverage of weather variables across all areas of the United States.  

There is a github site to help researchers work with the Daymet dataset and there is a python script that can be used to help extract and download multiple, single grid cells.  The site can be found here: https://github.com/ornldaac/daymet-single-pixel-batch/tree/master/python

We downloaded the file (daymet_multiple_extraction.py) and saved it in the daymet_files workspace folder. Following the example, we need to create a txt file with a specific format to tell the daymet_multiple_extracton.py script which pixels we want data for. We only need coordinate latlon pairs and filenames to download all variables and years, but we can specify variables and years if necessary.

Since we've made some changes with our main phen_base dataframe, we should check what Site_IDs we will need to download weather data.

In [ ]:
#How many unique site locations are there? How many unique plant ids are there? 

#number of locations 
print("number of unique locations:", len(phen_base.groupby(['Latitude', 'Longitude']).count()))
#number of plants
print("number of unique plants:", len(phen_base.groupby(['Individual_ID']).count())) #636
#number of sites
print("number of unique sites:", len(pd.unique(phen_base['Site_ID']))) 

number of unique locations: 410
number of unique plants: 505
number of unique sites: 412


We've decreased the number of sites and individual plant ids.  With the number of unique sites at 410 and the number of unique locations at 412 it looks like there are 2 latlons that have 2 sites at each.

We can now create the necessary txt file for the daymet_multiple_extraction.py script to run.

In [ ]:
#Need to get a text file (with csv format) that includes the latitude, longitude, and csv filenames.

#find updated unique latlon locations
latlon_df = phen_base.drop_duplicates(subset=['Latitude', 'Longitude', 'Site_ID'])[['Latitude', 'Longitude', 'Site_ID']]

#want to add a column for filename at the front of the lat lon
latlon_df['f4daymet']="site"+latlon_df['Site_ID'].astype(str)+".csv"

#correct order and values
latlon = latlon_df[['f4daymet', 'Latitude', 'Longitude']]

#export txt file (for use in below code)
latlon.to_csv("daymet_files/latlon.txt", index=False, header=False)

Run the below commands in the terminal, to initiate code (daymet_multiple_extractoin.py), get the appropriate data daymet tile based on latlon (referenced in the latlon.txt file) and output each siteid as its own csv file that contains all daymet data for all years 1980-2020 and all variables.

>cd daymet_files

>python daymet_multiple_extraction.py latlon.txt

## 2.2 Import Weather Data

The above commands download 412 files, one for each Site_ID.  This will allow us to join the phenophase data to weather data via Site_ID.  Each csv file contains daily weather data for a 1km by 1km grid cell for the years 1980-2020.  The column descriptions are below (copied from https://daymet.ornl.gov/single-pixel-tool-guide)

year (no units): Year, repeated for each day in the year.

yday (no units): Integer representing day of year, values ranging from 1-365. NOTE: All Daymet years are 1 – 365 days, including leap years. The Daymet database includes leap-days. Values for December 31 are discarded from leap years to maintain a 365-day year so yday 365 = December 31 for non-leap years or December 30 for leap-years.

dayl (s/day): Duration of the daylight period for the day. This calculation is based on the period of the day during which the sun is above a hypothetical flat horizon.

prcp (mm/day): Daily total precipitation, sum of all forms converted to water-equivalent.

srad (W/m^2): Incident shortwave radiation flux density, taken as an average over the daylight period of the day. NOTE: Daily Total Radiation (MJ/m^2/day) can be calculated: ((srad (W/m^2) * dayl (s/day)) / l,000,000)

swe (kg/m^2): Snow water equivalent. The amount of water contained within the snowpack. 

tmax (deg c): Daily maximum 2-meter air temperature.

tmin (deg c): Daily minimum 2-meter air temperature.

vp (Pa): Water Vapor Pressure (in pascals). Daily average partial pressure of water vapor. (can derive mean relative humidity, mean absolute humidity, and mean heat index using vp)

We are going to create a single dataframe, containing all the weather variables, for the unsupervised learning tasks and another dataframe containing the rolling 30-day mean, for the supervised learning tasks. We chose the 30-day rolling mean as that is close to the average interval timeframe between first leaf and first bloom.  We also thought that would be a good window to capture the preceding weather variables to first leaf. We also chose to use mean vs median because we actually wanted to capture the impact of some of the larger extremes in our data. 

In [ ]:
#Merge all the individual csv files into one datframe and create the rolling mean
# Defining the working folder
folder_path = 'daymet_files'

# Getting all files in the folder
all_files = os.listdir(folder_path)

# Create a list to hold dataframes
dfs = []


# Filter out any subdirectories. in case we want to expand to more datasets*
all_files = [f for f in all_files if os.path.isfile(os.path.join(folder_path, f))]

# Iterate over each file and append to the dfs list
for file_name in all_files:
    file_path = os.path.join(folder_path, file_name)

    if file_path.startswith('daymet_files/site'):

        find = re.search(r'site(\d+).csv', file_path)
        site_id = find.group(1) 

        #See report and EDA for more information on why we chose the specific features
        df = pd.read_csv(file_path, skiprows=7).rename(columns={'yday':'DOY'})
        df['Site_ID'] = int(site_id)
        #df['avg_dayl'] = df['dayl (s)'].rolling(30).mean()
        df['avg_precip'] = df['prcp (mm/day)'].rolling(30).mean()
        df['avg_srad'] = df['srad (W/m^2)'].rolling(30).mean()
        #df['avg_swe'] = df['swe (kg/m^2)'].rolling(30).mean()
        df['avg_tmax'] = df['tmax (deg c)'].rolling(30).mean()
        df['avg_tmin'] = df['tmin (deg c)'].rolling(30).mean()
        # df['avg_vp'] = df['vp (Pa)'].rolling(30).mean()
    
        # Read each CSV file into a DataFrame and append to the list
        dfs.append(df)

# Concatenate all dataframes in the list into a single dataframe
weather = pd.concat(dfs, ignore_index=True)

#save csv for unsupervised learning tasks
#too large for it to save properly using Deepnote
#was created and saved on local machine and then uploaded to the files in Deepnote
#weather.to_csv('weather.csv', index=False)

# #create df to join to phenophase data for supervised learning tasks
weather = weather[['Site_ID', 'year', 'DOY', 'avg_precip', 'avg_tmax', 'avg_tmin', 'avg_srad']]
weather

,Site_ID,year,DOY,avg_precip,avg_tmax,avg_tmin,avg_srad
0,1007,1980,1,NaN,NaN,NaN,NaN
1,1007,1980,2,NaN,NaN,NaN,NaN
2,1007,1980,3,NaN,NaN,NaN,NaN
3,1007,1980,4,NaN,NaN,NaN,NaN
4,1007,1980,5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
6165575,9996,2020,361,4.995000,4.321000,-3.331333,136.659000
6165576,9996,2020,362,4.920333,4.005333,-3.527333,137.956667
6165577,9996,2020,363,4.946333,4.062333,-3.745667,142.073333
6165578,9996,2020,364,4.946333,3.898000,-3.886333,142.920667


Note: the first 30 days of the above dataframe will not have values in the average weather column as it needs 30 days of data to compute!

# 3. Merged Data

## 3.1 Phenophase Interval

Merges the interval phenotype and weather dataframes using site id, year, and DOY. This produces a dataframe that has the phenophases interval and the rolling 30 day average weather leading up to that day.

In [ ]:
#merge phenophase interval and weather data
#each row is data for a single plant, the interval between leaf and bloom, 
    #and the weather information on the day of first bloom at the site

df_interval = pd.merge(weather, phen_int, on=['Site_ID', 'year', 'DOY'])

#save csv
df_interval.to_csv('df_interval.csv', index=False)

df_interval

,Site_ID,year,DOY,avg_precip,avg_tmax,avg_tmin,avg_srad,Individual_ID,interval
0,10241,2015,140,1.171667,16.633000,4.686667,482.737667,39635,28.0
1,10329,2013,137,1.202667,20.590000,4.624000,432.779333,40131,22.0
2,10530,2013,144,3.402667,17.745667,5.513000,387.121000,41318,36.0
3,10596,2013,135,1.403000,15.766333,3.717000,436.880333,41754,25.0
4,10724,2013,140,2.025667,16.198333,3.628667,465.442000,42634,28.0
...,...,...,...,...,...,...,...,...,...
117,867,2014,129,5.441000,18.852333,5.048333,421.161000,5121,8.0
118,952,2009,126,4.497667,14.436667,5.383000,355.808000,943,24.0
119,974,2009,121,3.858667,16.412333,3.724667,399.633667,975,19.0
120,9910,2013,126,2.569667,16.971000,4.380000,402.821333,37740,25.0


## 3.2 Phenophase Presence

Merges the phenotype and weather dataframes using site id, year, and DOY. This produces a dataframe that has a phenophases DOY and weather data for years that have phenophase data, but keeps all days in that year. For this task, we are using the weather information at the site to predict the day of first bloom and day of first leaf.  

We should only keep weather data for years that have pheno data, but we want all days in that year
1. pull unique Site ID and years from pheno
2. slice weather to only those Site IDs and years using right join of weather on years
3. then left merge step 2 on phen

In [ ]:
# Logistic and Random Forest 
#reorganzing columns and slicing only to those needed to join to weather
phen_log = phen_base[['Site_ID', 'Latitude', 'Longitude', 'Individual_ID', 'year', 'DOY', 
            'Phenophase_Description']]

#adding in dummy variables
phen_log['value']=1
phen_log = phen_log.pivot(index = ['Site_ID', 'Latitude', 'Longitude', 'Individual_ID', 'year', 'DOY'], 
    columns=['Phenophase_Description'], values=['value']).reset_index()
phen_log = phen_log.fillna(0)
phen_log['First_leaf'] = phen_log.loc[:, ('value',['First leaf'])]
phen_log['First_bloom'] = phen_log.loc[:, ('value',['First bloom'])]
phen_log.columns = phen_log.columns.droplevel(1)
phen_log['Phenophase'] = phen_log['First_bloom'].apply(lambda x: x+1 if x==1 else 0)
phen_log['Phenophase'] = phen_log['Phenophase'] + phen_log['First_leaf']

#reorganzing columns and slicing only to those needed to join to weather
phen_log = phen_log[['Site_ID', 'Latitude', 'Longitude', 'Individual_ID', 'year', 'DOY', 
            'First_leaf', 'First_bloom', 'Phenophase']]

phen_log

/tmp/ipykernel_40/3202211780.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Site_ID,Latitude,Longitude,Individual_ID,year,DOY,First_leaf,First_bloom,Phenophase
0,2,43.085350,-70.691330,521,2009,106,1.0,0.0,1.0
1,2,43.085350,-70.691330,521,2009,140,0.0,1.0,2.0
2,2,43.085350,-70.691330,521,2010,96,1.0,0.0,1.0
3,2,43.085350,-70.691330,521,2010,125,0.0,1.0,2.0
4,2,43.085350,-70.691330,521,2011,112,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
3782,38489,39.065994,-76.504898,237790,2020,113,0.0,1.0,2.0
3783,38569,41.034615,-73.740723,238151,2020,116,0.0,1.0,2.0
3784,38658,44.145035,-68.851357,239777,2020,120,1.0,0.0,1.0
3785,38956,41.802170,-70.556267,242065,2020,139,0.0,1.0,2.0


In [ ]:
years = phen_base[['year', 'Site_ID']].drop_duplicates()
merged_df = weather.merge(years, how='right', on=['Site_ID', 'year']).dropna()
merged_df

,Site_ID,year,DOY,avg_precip,avg_tmax,avg_tmin,avg_srad
29,209,1980,30,0.894000,-0.962333,-12.123333,206.480000
30,209,1980,31,0.894000,-1.317000,-12.430000,207.782333
31,209,1980,32,0.894000,-1.607333,-12.810333,211.193000
32,209,1980,33,0.894000,-1.803333,-12.977000,212.198333
33,209,1980,34,0.894000,-1.938333,-12.965333,211.908000
...,...,...,...,...,...,...,...
688020,39049,2020,361,6.833667,7.903667,1.311333,138.872000
688021,39049,2020,362,6.833667,7.584333,0.934333,140.276667
688022,39049,2020,363,6.833667,7.458667,0.627333,143.195333
688023,39049,2020,364,6.833667,7.322000,0.557667,142.423333


In [ ]:
#merge pheno and weather data
merged_df = merged_df.merge(phen_log, how='left', on=['Site_ID', 'year', 'DOY'])
merged_df

,Site_ID,year,DOY,avg_precip,avg_tmax,avg_tmin,avg_srad,Latitude,Longitude,Individual_ID,First_leaf,First_bloom,Phenophase
0,209,1980,30,0.894000,-0.962333,-12.123333,206.480000,NaN,NaN,NaN,NaN,NaN,NaN
1,209,1980,31,0.894000,-1.317000,-12.430000,207.782333,NaN,NaN,NaN,NaN,NaN,NaN
2,209,1980,32,0.894000,-1.607333,-12.810333,211.193000,NaN,NaN,NaN,NaN,NaN,NaN
3,209,1980,33,0.894000,-1.803333,-12.977000,212.198333,NaN,NaN,NaN,NaN,NaN,NaN
4,209,1980,34,0.894000,-1.938333,-12.965333,211.908000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
684137,39049,2020,361,6.833667,7.903667,1.311333,138.872000,NaN,NaN,NaN,NaN,NaN,NaN
684138,39049,2020,362,6.833667,7.584333,0.934333,140.276667,NaN,NaN,NaN,NaN,NaN,NaN
684139,39049,2020,363,6.833667,7.458667,0.627333,143.195333,NaN,NaN,NaN,NaN,NaN,NaN
684140,39049,2020,364,6.833667,7.322000,0.557667,142.423333,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_df = merged_df[['First_bloom', 'First_leaf', 'Phenophase', 'avg_precip', 'avg_tmax', 'avg_tmin', 'avg_srad']]
merged_df = merged_df.fillna(0)
merged_df

,First_bloom,First_leaf,Phenophase,avg_precip,avg_tmax,avg_tmin,avg_srad
0,0.0,0.0,0.0,0.894000,-0.962333,-12.123333,206.480000
1,0.0,0.0,0.0,0.894000,-1.317000,-12.430000,207.782333
2,0.0,0.0,0.0,0.894000,-1.607333,-12.810333,211.193000
3,0.0,0.0,0.0,0.894000,-1.803333,-12.977000,212.198333
4,0.0,0.0,0.0,0.894000,-1.938333,-12.965333,211.908000
...,...,...,...,...,...,...,...
684137,0.0,0.0,0.0,6.833667,7.903667,1.311333,138.872000
684138,0.0,0.0,0.0,6.833667,7.584333,0.934333,140.276667
684139,0.0,0.0,0.0,6.833667,7.458667,0.627333,143.195333
684140,0.0,0.0,0.0,6.833667,7.322000,0.557667,142.423333


In [ ]:
#save csv
#too large for it to save properly using Deepnote
#was created and saved on local machine and then uploaded to the files in Deepnote
#merged_df.to_csv('df_phen_presence.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6bbd9398-18f2-46ef-ba4f-be9bc7f4116a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>